# 한국 IPv4 관리대행자별 대역 분리

엑셀 파일 다운로드: https://한국인터넷정보센터.한국/jsp/business/management/isCurrentIpv4.jsp

In [1]:
import math
import pandas as pd

In [2]:
df = pd.read_csv('KR-IPv4.csv')
df.columns = ['Organization', 'ServiceName', 'Start', 'End', 'Count24', 'Date']
df.drop(columns=['Date'], inplace=True)

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3281 entries, 0 to 3280
Data columns (total 5 columns):
Organization    3281 non-null object
ServiceName     3281 non-null object
Start           3281 non-null object
End             3281 non-null object
Count24         3281 non-null int64
dtypes: int64(1), object(4)
memory usage: 128.2+ KB
None


,Organization,ServiceName,Start,End,Count24
0,(주)아프리카티비,AFREECATV,203.238.128.0,203.238.159.255,32
1,(주)엘지유플러스,BORANET,210.98.224.0,210.98.227.255,4
2,(주)엘지유플러스,BORANET,210.98.228.0,210.98.231.255,4
3,(주)엘지유플러스,BORANET,210.98.232.0,210.98.235.255,4
4,(주)엘지유플러스,BORANET,210.98.236.0,210.98.239.255,4


In [3]:
# def test(x):
#     return math.log2(x['Count24']) % 1 != 0
# df = df[df.apply(test, axis=1)]
# df.head()

In [4]:
def ipStrToInt(ip):
    weight = [2**24, 2**16, 2**8, 2**0]
    mul = lambda x: int(x[0]) * x[1]
    return sum(map(mul, zip(ip.split('.'), weight)))

def ipIntToStr(ip):
    weight = [2**24, 2**16, 2**8, 2**0]
    div = lambda x: str(int((x[0] % x[1]) / x[2]))
    return '.'.join(map(div, zip([ip]*len(weight), [2**32]+weight, weight)))

assert ipStrToInt('123.45.67.89') == 2066563929
assert ipIntToStr(2066563929) == '123.45.67.89'

In [5]:
df['IntStart'] = df['Start'].map(ipStrToInt)
df['IntEnd'] = df['End'].map(ipStrToInt)
df['Count'] = df['IntEnd'] - df['IntStart'] + 1
df.head()    

,Organization,ServiceName,Start,End,Count24,IntStart,IntEnd,Count
0,(주)아프리카티비,AFREECATV,203.238.128.0,203.238.159.255,32,3421405184,3421413375,8192
1,(주)엘지유플러스,BORANET,210.98.224.0,210.98.227.255,4,3529695232,3529696255,1024
2,(주)엘지유플러스,BORANET,210.98.228.0,210.98.231.255,4,3529696256,3529697279,1024
3,(주)엘지유플러스,BORANET,210.98.232.0,210.98.235.255,4,3529697280,3529698303,1024
4,(주)엘지유플러스,BORANET,210.98.236.0,210.98.239.255,4,3529698304,3529699327,1024


In [6]:
for index, row in df.iterrows():
    assert row['Count24'] * (2**8) == row['Count'], row

In [7]:
def countToPrefixArr(count):
    result = []
    while count:
        maxPrefix = 2 ** int(math.log2(count))
        count -= maxPrefix
        result.append(maxPrefix)
    return result

In [8]:
df['Prefix'] = ''
df['Range'] = ''

table = []
for index, row in df.iterrows():
    prefixes = countToPrefixArr(row['Count24'])
    start = row['IntStart']
    end = 0
    
    for prefix in prefixes:
        newRow = row.to_dict()
        newRow['Count24'] = prefix
        newRow['Count'] = prefix * (2**8)
        newRow['Start'] = ipIntToStr(start)
        newRow['IntStart'] = start
        end = start + newRow['Count'] - 1
        start = end + 1
        newRow['End'] = ipIntToStr(end)
        newRow['IntEnd'] = end
        newRow['Prefix'] = '/' + str(32 - int(math.log2(newRow['Count'])))
        newRow['Range'] = newRow['Start'] + newRow['Prefix']
        table.append(newRow)
        
df = pd.DataFrame(table, columns=df.columns)
df.head(10)

,Organization,ServiceName,Start,End,Count24,IntStart,IntEnd,Count,Prefix,Range
0,(주)아프리카티비,AFREECATV,203.238.128.0,203.238.159.255,32,3421405184,3421413375,8192,/19,203.238.128.0/19
1,(주)엘지유플러스,BORANET,210.98.224.0,210.98.227.255,4,3529695232,3529696255,1024,/22,210.98.224.0/22
2,(주)엘지유플러스,BORANET,210.98.228.0,210.98.231.255,4,3529696256,3529697279,1024,/22,210.98.228.0/22
3,(주)엘지유플러스,BORANET,210.98.232.0,210.98.235.255,4,3529697280,3529698303,1024,/22,210.98.232.0/22
4,(주)엘지유플러스,BORANET,210.98.236.0,210.98.239.255,4,3529698304,3529699327,1024,/22,210.98.236.0/22
5,(주)엘지유플러스,BORANET,61.97.32.0,61.97.47.255,16,1029775360,1029779455,4096,/20,61.97.32.0/20
6,(주)엘지유플러스,BORANET,61.97.112.0,61.97.127.255,16,1029795840,1029799935,4096,/20,61.97.112.0/20
7,(주)엘지유플러스,BORANET,61.97.48.0,61.97.63.255,16,1029779456,1029783551,4096,/20,61.97.48.0/20
8,(주)엘지유플러스,BORANET,211.172.144.0,211.172.159.255,16,3551301632,3551305727,4096,/20,211.172.144.0/20
9,(주)엘지유플러스,BORANET,211.36.128.0,211.36.159.255,32,3542384640,3542392831,8192,/19,211.36.128.0/19


In [9]:
for index, row in df.iterrows():
    assert row['IntEnd'] - row['IntStart'] + 1 == row['Count'], row

In [10]:
df.to_excel('final.xlsx', index=False)

+ /24 Count : Prefix
+ 1 : /24
+ 2 : /23
+ 4 : /22
+ 8 : /21
+ 16 : /20
+ 32 : /19
+ 64 : /18
+ 128 : /17
+ 256 : /16
+ ...